Evaluation of Llama not Finetuned

In [2]:
!pip install fsspec==2024.10.0
!pip cache purge

Files removed: 0


In [1]:
!pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes
! pip install -q -U rouge_score
! pip install -q -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 29.6 MB/s eta 0:00:00


In [2]:
!pip install evaluate
from evaluate import load
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00


### Imports

In [3]:
import pandas as pd
import json
import torch
import os
import time

from datasets import load_dataset, Dataset, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print

from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

import textwrap

### Global Constants

In [4]:
model_id = "TheBloke/Llama-2-7B-GPTQ"

In [5]:
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"
LLMRES_PKL_FILE="sql_test_Llama2.pkl"

### Common Functions

In [7]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [8]:
rouge = load("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

In [10]:
def gen_eval_res(test_df):
  eval_res = []
  time_res = []
  row_no = 1
  for row in test_df.itertuples(index=False):
    start_time = perf_counter()
    text = row.text
    llm_res = pipe(text)[0]['generated_text']
    parse_res = parse(llm_res)
    end_time = perf_counter()
    eval_res.append(parse_res)
    time_res.append(end_time-start_time)
    if row_no % 10 == 0:
      print(f"*** Row {row_no} Done ***")
    row_no += 1

  test_df['eval_res'] = eval_res
  test_df['eval_time'] = time_res

### Load and Check Data

In [11]:
dataset = load_from_disk(DATA_PATH + DS_DIR)

In [12]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [13]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [35]:
display(test_df.head(2))
display(test_df.shape)

,response,question,context,__index_level_0__,text,eval_res,eval_time
0,"SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...",429,### Instruction:\n You are a powerful text-...,1. SELECT COUNT(*) FROM race WHERE track_id = ...,4.275261
1,"SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...",2907,### Instruction:\n You are a powerful text-...,"SELECT s.shop_name AS 'Shop Name',s.shop_id as...",13.616137


(454, 7)

In [15]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
          model_id,
          quantization_config=quantization_config_loading,
          device_map="auto"
        )

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Llama-2-7B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bia

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [16]:
print(model.get_memory_footprint()/1e9)

3.893870848

In [17]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=180,
    do_sample = True,
    temperature=0.1,
    top_k=4,
    repetition_penalty=1.2
)

llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-17-04883f48fc6a>:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


#### Check Model on Sample Test Question

In [18]:
ID = 10

In [19]:
test_df['text'][ID]

'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    Show ids for all students who have advisor 1121.\n    ### Context:\n    CREATE TABLE Student (StuID VARCHAR, Advisor VARCHAR)\n    ### Response:\n    '

In [20]:
display(test_df['response'][ID])

'SELECT StuID FROM Student WHERE Advisor = 1121'

In [21]:
text = test_df['text'][ID]
llm_res = pipe(text)[0]['generated_text']
parse_res = parse(llm_res)
print(parse_res)

SELECT StuId FROM student WHERE Advisor = '1121'
### Explanation:
    The above example shows how you can use your powers of natural language processing to understand what it means 
when someone asks "Show me all students with an advisor named 1121."

In [22]:
eval_start_time = perf_counter()
gen_eval_res(test_df)
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


*** Row 10 Done ***

*** Row 20 Done ***

*** Row 30 Done ***

*** Row 40 Done ***

...

*** Row 420 Done ***

*** Row 430 Done ***

*** Row 440 Done ***

*** Row 450 Done ***

Evaluation Took:  4337.48

In [31]:
import os
DATA_PATH = "/content/drive/MyDrive/Text2SQL/Data/"
PKL_DIR = "test/"
LLMRES_PKL_FILE = "sql_test_Llama2.pkl"
target_dir = os.path.join(DATA_PATH, PKL_DIR)
os.makedirs(target_dir, exist_ok=True)
file_path = os.path.join(target_dir, LLMRES_PKL_FILE)
with open(file_path, "wb") as f:
    pass

Empty file sql_test_Llama2.pkl successfully created at 
/content/drive/MyDrive/Text2SQL/Data/test/sql_test_Llama2.pkl

In [32]:
test_df.to_pickle('/content/drive/MyDrive/Text2SQL/Data/test/sql_test_Llama2.pkl')

In [34]:
import os

file_path = "/content/drive/MyDrive/Text2SQL/Data/test/sql_test_Llama2.pkl"
if os.path.exists(file_path):
    print(f"File exists: {file_path}")
    print(f"File size: {os.path.getsize(file_path)} bytes")
else:
    print(f"File not found: {file_path}")


File exists: /content/drive/MyDrive/Text2SQL/Data/test/sql_test_Llama2.pkl

File size: 437829 bytes

In [33]:
test_df2=pd.read_pickle('/content/drive/MyDrive/Text2SQL/Data/test/sql_test_Llama2.pkl')
display(test_df2.head(2))

,response,question,context,__index_level_0__,text,eval_res,eval_time
0,"SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...",429,### Instruction:\n You are a powerful text-...,1. SELECT COUNT(*) FROM race WHERE track_id = ...,4.275261
1,"SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...",2907,### Instruction:\n You are a powerful text-...,"SELECT s.shop_name AS 'Shop Name',s.shop_id as...",13.616137
